In [1]:
# Classifier imports
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.metrics import classification_report
from sklearn.metrics import RocCurveDisplay


import pandas as pd
import numpy as np

from joblib import dump, load

import os
os.getcwd()

'C:\\Users\\mlabuss\\OneDrive - UvA\\CAREER\\SICSS_Odissei\\SICSS-ODISSEI\\fertility-prediction-challenge'

# 1. Read data

In [2]:
# Import paths
path_train = "./data/LISS_example_input_data.csv"
path_outcome = "./data/LISS_example_groundtruth_data.csv"


In [5]:
# Read data
original_data = pd.read_csv(path_train, encoding="cp1252", low_memory=False)
outcome = pd.read_csv(path_outcome, encoding="cp1252", low_memory=False)

## Feature selection

In [20]:
#original_data.loc[:,['leeftijd2019']]

#filter_col = [col for col in original_data if col.startswith('cf19l401')]
#filter_col

['cf19l401']

In [6]:
keepcols = [
    # Background characteristics
          'leeftijd2019',
          'geslacht',
          'positie2019',
          'aantalhh2019',
          'aantalki2019',
          'partner2019',
          'burgstat2019',
          'woonvorm2019',
          'woning2019',
          'sted2019',
          'belbezig2019',
          'brutoink2019',
          'brutoink_f2019',
          'netinc2019',
          'brutohh_f2019',
          'nettohh_f2019',
          'oplzon2019',
          'oplmet2019',
          'doetmee2019',
          'herkomstgroep2019',
          'simpc2019',
          
    # Family
          'cf19l401', # relationship with family
          'cf19l432', # whether your parents are still together
          'cf19l032', # gender of partner 
          'cf19l454', # have any children
          'cf19l455', # number of living children
          'cf19l068', # gender of children
          'cf19l069',                          
          'cf19l070',
          'cf19l071',
          'cf19l456', # age of children 
          'cf19l457',
          'cf19l458',
          'cf19l459',
          'cf19l129', # how many children in the future
          'cf19l130', # Within how many years do you hope to have your
          'cf19l408', # attitude towards not having kids                         
          'cf19l483', # gender household division
          'cf19l484',                         
          'cf19l485',                       
          'cf19l486',
          'cf19l487',
          'cf19l488',
    
    # Work
          'cw19l001', # Does respondent have paid work?
          'cw19l006', # What then is the highest level of education that you have completed with diploma or certificate?
          'cw19l008', # the highest level of education that you have attended
          'ci19l252', # How would you describe the financial situation of your household at this moment?
          'ci19l339',  #What was total net income of your household over the period from 1 January 2018 to 31 December 2018?
          'cd19l034', # How many rooms does your dwelling contain?
    
    # Health & religion
          'ch19l085', #  diabitis
          'ch19l004', # self rated health
          'cr19l042', # pray
          'cr19l143' # religous affiliation
                                     ]
data_selection = original_data.loc[:,keepcols]
data_selection.shape

(9459, 53)

In [5]:
#outcome['new_child'].value_counts(dropna=False)

NaN    8557
0.0     753
1.0     149
Name: new_child, dtype: int64

In [7]:
# Drop observations where the outcome is missing
y_isna = outcome['new_child'].isnull()
data = data_selection.loc[~y_isna]
outcome = outcome.loc[~y_isna]

print(data.shape, outcome.shape)

(902, 53) (902, 2)


## Feature missing values

In [8]:
data_wmiss = data.dropna(axis=1, thresh=0.7*len(data))
data_wmiss.shape
#listdata_wmiss.columns.to_list()

(902, 28)

# 2. Split data into train and test
First thing always, otherwise you risk overfitting

In [9]:
# Select predictors: education, year of birth, gender, number of children in the household 
# You can do this automatically (not necessarily better): https://scikit-learn.org/stable/modules/feature_selection.html

X_train, X_test, y_train, y_test = train_test_split(data_wmiss,
                                                    outcome,
                                                    test_size=0.2, random_state=2023)
y_train = y_train["new_child"]
y_test = y_test["new_child"]

# 3. Pre-process and model
You may not want to include the preprocessing in the pipeline if it becomes too cumbersome

Make sure to use the scoring that you want to optimize in the search space

In [10]:
#X_train.dtypes

In [11]:
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include = object)
categorical_columns = categorical_columns_selector(data_wmiss)

numerical_columns_selector = selector(dtype_include = "float64")
numerical_columns = numerical_columns_selector(data_wmiss)

print(numerical_columns)
print(categorical_columns)

['leeftijd2019', 'brutoink_f2019', 'netinc2019', 'brutohh_f2019', 'nettohh_f2019', 'cf19l432', 'cf19l454', 'cw19l001', 'cw19l008', 'ch19l085', 'ch19l004', 'cr19l042', 'cr19l143']
['geslacht', 'positie2019', 'aantalhh2019', 'aantalki2019', 'partner2019', 'burgstat2019', 'woonvorm2019', 'woning2019', 'sted2019', 'belbezig2019', 'oplzon2019', 'oplmet2019', 'doetmee2019', 'herkomstgroep2019', 'simpc2019']


In [12]:
# An example of a preprocessing apart from the pipeline
#dict_kids = {'None': 0, 'One child': 1, 'Two children': 2, 'Three children': 3, 'Four children': 4, 'Five children': 5, 'Six children': 6}
#X_train["aantalki2019"] = X_train["aantalki2019"].map(dict_kids)

# Create transformers
# Imputer are sometimes not necessary
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='infrequent_if_exist', min_frequency=50))])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy="mean")),
    ('scaler', StandardScaler())])

# Use ColumnTransformer to apply the transformations to the correct columns in the dataframe
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, selector(dtype_exclude=object)(X_train)),
        ('cat', categorical_transformer, selector(dtype_include=object)(X_train))])

## Model 1: logistic regression 

In [13]:
# Create pipeline
model_1 = Pipeline([
               ("preprocess", preprocessor),
               ("classifier", LogisticRegression(max_iter=500))
               ]) 
                    

In [ ]:
# Define the hyperparameters, this can include several classifiers, but will make it slow
# You can see different classifiers here: https://scikit-learn.org/stable/supervised_learning.html#supervised-learning
parameters = [
    {
        'classifier': [LogisticRegression(max_iter=500)],
        'classifier__C': np.logspace(-3, 3, 50) #regularization coefficient
    },
    
]

# Perform hyperparameter tuning using cross-validation: https://scikit-learn.org/stable/modules/classes.html#hyper-parameter-optimizers
# Scoring metrics: https://scikit-learn.org/stable/modules/model_evaluation.html
# f1 = f1 of the class labeled as 1 (i.e. kids)
grid_search = GridSearchCV(model_1, parameters, cv=5, n_jobs=-1, scoring="f1", verbose=9) #n_jobs=-1 allows for multiprocessing
grid_search.fit(X_train, y_train)

# Keep best model (or define it from scratch with the best coefficients found)
best_model_1 = grid_search.best_estimator_

best_model_1

In [ ]:
#Variable names in the data
#best_model["preprocess"].get_feature_names_out()

In [ ]:
model_1.fit(X_train,y_train)


In [ ]:
prediction = model_1.predict(X_test)
model_1.score(X_test,y_test)

### Cross-validation

In [ ]:
from sklearn.model_selection import cross_validate
cross_validate(model_1, X_train, y_train.values.ravel(), cv=5)

### Best model from GridSearch

In [ ]:
best_model_1.fit(X_train,y_train)
best_prediction = best_model_1.predict(X_test)
best_model_1.score(X_test,y_test)

## Models 2 and 3: Random forests
### Simple random forests

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

model_2 = Pipeline([
               ("preprocess", preprocessor),
               ("classifier", RandomForestClassifier(n_estimators=100, random_state=456))
               ]) 

cross_validate(model_2, X_test, y_test, cv=5)

{'fit_time': array([0.29484415, 0.29949164, 0.28100252, 0.28249407, 0.27795196]),
 'score_time': array([0.02890182, 0.02592397, 0.02627039, 0.02625346, 0.02483559]),
 'test_score': array([0.89189189, 0.83333333, 0.88888889, 0.86111111, 0.83333333])}

### Extremely random forests

In [26]:
from sklearn.ensemble import ExtraTreesClassifier
extra_tree = ExtraTreesClassifier(n_estimators=100, criterion='gini', 
                           max_depth=None, min_samples_split=2, 
                           min_samples_leaf=1,  max_features='sqrt', 
                           bootstrap=False, random_state=345)

model_3 = Pipeline([
               ("preprocess", preprocessor),
               ("classifier", extra_tree)
               ]) 

cross_validate(model_3, X_test, y_test, cv=5)

{'fit_time': array([2.08198428, 2.13250208, 2.03405404, 2.01729941, 2.06727386]),
 'score_time': array([0.16081667, 0.1626482 , 0.16189504, 0.16094446, 0.16128445]),
 'test_score': array([0.89189189, 0.83333333, 0.91666667, 0.88888889, 0.86111111])}

In [42]:
#model_3.get_params()
from pprint import pprint
pprint(model_3.get_params())

'classifier__bootstrap': False,
 'classifier__ccp_alpha': 0.0,
 'classifier__class_weight': None,
 'classifier__criterion': 'gini',
 'classifier__max_depth': None,
 'classifier__max_features': 'sqrt',
 'classifier__max_leaf_nodes': None,
 'classifier__max_samples': None,
 'classifier__min_impurity_decrease': 0.0,
 'classifier__min_samples_leaf': 1,
 'classifier__min_samples_split': 2,
 'classifier__min_weight_fraction_leaf': 0.0,
 'classifier__n_estimators': 1000,
 'classifier__n_jobs': None,
 'classifier__oob_score': False,
 'classifier__random_state': 345,
 'classifier__verbose': 0,
 'classifier__warm_start': False}

**Number of trees in random forest**
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
**Number of features to consider at every split**
max_features = ['auto', 'sqrt']
**Maximum number of levels in tree**
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
**Minimum number of samples required to split a node**
min_samples_split = [2, 5, 10]
**inimum number of samples required at each leaf node**
min_samples_leaf = [1, 2, 4]
**Method of selecting samples for training each tree**
bootstrap = [True, False]

See https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

None


In [46]:
# Define the hyperparameters, this can include several classifiers, but will make it slow
# You can see different classifiers here: https://scikit-learn.org/stable/supervised_learning.html#supervised-learnin

list_max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
list_max_depth.append(None)

parameters = {
    "classifier__n_estimators": [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)], 
    "classifier__max_depth": list_max_depth,
    'classifier__criterion': ['gini','entropy'],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__min_samples_split': [2, 5, 10]
}

# Perform hyperparameter tuning using cross-validation: https://scikit-learn.org/stable/modules/classes.html#hyper-parameter-optimizers
grid_search = GridSearchCV(model_3, parameters, cv=3, n_jobs=-1, scoring="f1", verbose=9) #n_jobs=-1 allows for multiprocessing
grid_search.fit(X_train, y_train)

# Keep best model (or define it from scratch with the best coefficients found)
best_model_3 = grid_search.best_estimator_

best_model_3

Fitting 3 folds for each of 2160 candidates, totalling 6480 fits


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['leeftijd2019',
                                                   'brutoink_f2019',
                                                   'netinc2019',
                                                   'brutohh_f2019',
                                                   'nettohh_f2019', 'cf19l432',
                                                   'cf19l454', 'cw19l001',
                                                   'cw19l008', 'ch19l085',
                                                   'ch19l004', 'cr19l042',
                                                   'cr19l143']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   Simpl...
                                                                   OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                                 min_frequency=50))]),
                                                  ['geslacht', 'positie2019',
                                                   'aantalhh2019',
                                                   'aantalki2019',
                                                   'partner2019',
                                                   'burgstat2019',
                                                   'woonvorm2019', 'woning2019',
                                                   'sted2019', 'belbezig2019',
                                                   'oplzon2019', 'oplmet2019',
                                                   'doetmee2019',
                                                   'herkomstgroep2019',
                                                   'simpc2019'])])),
                ('classifier',
                 ExtraTreesClassifier(max_depth=30, n_estimators=800,
                                      random_state=345))])

In [47]:
best_model_3.fit(X_train,y_train)
best_prediction = best_model_3.predict(X_test)
best_model_3.score(X_test,y_test)
#0.88

0.8839779005524862

# Evaluate the model

Note: The results below are not for LogisticRegression, are for a different model

In [ ]:
# Print ROC curve, it tells you how well you can balance false and true positives
RocCurveDisplay.from_predictions(
    y_test,
    model_1.predict_proba(X_test)[:, 1],
    color="cornflowerblue",
)
plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

In [ ]:
RocCurveDisplay.from_predictions(
    y_test,
    model_3.predict_proba(X_test)[:, 1],
    color="cornflowerblue",
)
plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

In [ ]:
RocCurveDisplay.from_predictions(
    y_test,
    best_model_3.predict_proba(X_test)[:, 1],
    color="cornflowerblue",
)
plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

In [ ]:
# Create predictions
y_pred = best_model_1.predict(X_test)

# Report classification table
print(classification_report(y_test, y_pred))

In [ ]:
# Create predictions
y_pred = best_model_3.predict(X_test)

# Report classification table
print(classification_report(y_test, y_pred))

# Save models 

In [ ]:
import os
os.makedirs("../models", exist_ok=True)

# Dump model (don't change the name)
dump(best_model_1, "../models/model.joblib")

# How the submission would look like, 

In [ ]:
def predict_outcomes(df):
    """Process the input data and write the predictions."""

    # Keep 
    keepcols =  ['leeftijd2019',
                 'geslacht',
                 'positie2019',
                 'aantalhh2019',
                 'aantalki2019',
                 'partner2019',
                 'burgstat2019',
                 'woonvorm2019',
                 'woning2019',
                 'sted2019',
                 'belbezig2019',
                 'brutoink_f2019',
                 'netinc2019',
                 'brutohh_f2019',
                 'nettohh_f2019',
                 'oplzon2019',
                 'oplmet2019',
                 'doetmee2019',
                 'herkomstgroep2019',
                 'simpc2019',
                 'cf19l432',
                 'cf19l454',
                 'cw19l001',
                 'cw19l008',
                 'ch19l085',
                 'ch19l004',
                 'cr19l042',
                 'cr19l143']
    results = df[["nomem_encr"]]
    
    df = df.loc[:, keepcols]
                            
    # Load your trained model from the models directory
    model_path = os.path.join(os.path.dirname(__file__), "..", "models", "model.joblib")
    model = load(model_path)

    # Use your trained model for prediction
    results.loc[:, "prediction"] = model.predict(df)

    #If you use predict_proba to get a probability and a different threshold
    #df["prediction"] = (df["prediction"] >= 0.5).astype(int)
    return results

In [ ]:
__file__ = "./"
predict_outcomes(original_data)

In [ ]:
__file__ = './' #this is not needed outside juypter notebooks
predict_outcomes(original_data)

# Interpretable AI

Note: Again, the result below are not for the LogisticRegression

There are other methods 
    - tree-based models have an argument "best_model.feature_importances_"
    - Other libraries: shap, lime, eli5
    - Other plots, such as partial dependence plots: https://scikit-learn.org/stable/auto_examples/inspection/plot_partial_dependence.html#sphx-glr-auto-examples-inspection-plot-partial-dependence-py

In [ ]:
from sklearn.inspection import permutation_importance

def print_sorted_importance(importances, columns, codebook=None):
    for imp, var in sorted(zip(importances, columns))[::-1]:
        if codebook is not None:
            print(f"{imp:2.3f}: {codebook[var]:50.50s}")
        else:
            print(f"{imp:2.3f}: {var:50.50s}")
        
r = permutation_importance(best_model_1, X_test, y_test,
                            n_repeats=100,
                            random_state=0)

print_sorted_importance(r["importances_mean"], X_test.columns)

In [ ]:
sorted(list(zip(best_model_1["classifier"].coef_[0], best_model_1["preprocess"].get_feature_names_out())))